In [1]:
import json
import random
from special_moves import SPECIAL_FUNCTIONS

# LOAD FILES
with open("Moves.json", "r") as f:
    MOVESET = json.load(f)

with open("Types.json", "r") as f:
    TYPES = json.load(f)

In [2]:
# MOVESET
moveset = {
    1: MOVESET["Heal"],
    2: MOVESET["Attack"],
    3: MOVESET["Counter-Attack"],
    4: MOVESET["Lifesteal"],
}

In [3]:
# GAME START AND INFO
def state_start(data):
    is_first = random.choice([True, False])
    p_type = TYPES[str(random.choice([0, 2]))]
    e_type = TYPES[str(random.choice([1, 1]))]

    data.update({
        "is_player_first": is_first,
        "last_actor": None,

        "player_hp": p_type["base_hp"],
        "player_type": p_type,
        "player_move": None,

        "enemy_hp": e_type["base_hp"],
        "enemy_type": e_type,
        "enemy_move_type": None
    })

    print("=== BATTLE STARTING ===")
    print(("PLAYER" if is_first else "ENEMY"), "goes first!\n")
    return "DISPLAY_INFO"


def state_display_info(data):
    print("--PLAYER INFO--")
    print("Health:", data["player_hp"])
    print("Type:", data["player_type"]["name"])
    print("\n--ENEMY INFO--")
    print("Health:", data["enemy_hp"])
    print("Type:", data["enemy_type"]["name"])
    print()

    return "INPUT_PLAYER"

In [4]:
# GET PLAYER MOVE SELECTION
def state_input_player(data):
    move_index = int(input("Select a move (1-4): "))
    move = moveset.get(move_index)
    if move is None:
        print("Invalid move! Turn wasted.")
        data['last_actor'] = 'player'
    
    data['player_move'] = move
    return "PLAYER_TURN" if data['is_player_first'] else "ENEMY_TURN"

In [5]:
def get_type_damage_modifier(data, actor):
    EFFECTIVE = 1.2
    INEFFECTIVE = 0.8

    # actor = attacker
    if actor == "player":
        attacker = data["player_type"]
        defender = data["enemy_type"]
    else:
        attacker = data["enemy_type"]
        defender = data["player_type"]

    # comparing types

    if attacker == defender:
        return 1.0

    if defender == TYPES[attacker["effective"]]:
        return EFFECTIVE

    if defender == TYPES[attacker["ineffective"]]:
        return INEFFECTIVE

    return 1.0

In [6]:
# PLAYER TURN
def state_player_turn(data):
    print("=== PLAYER TURN ===")
    move = data['player_move']
    if move == None:
        return "CHECK_WIN" 
    
    print("YOU used:", move["type"])
    ptype = data['player_type']

    # HANDLE HP
    if move["type"] == "HEAL":
        data["player_hp"] += ptype['heal_base_hp']

    elif move["type"] == "ATTACK":
        data["enemy_hp"] -= ptype['attack_base_hp'] * get_type_damage_modifier(data, 'player')

    elif move["type"] == "SPECIAL":
        print(SPECIAL_FUNCTIONS[move["function"]](data))
        

    data['last_actor'] = 'player'
    return "CHECK_WIN"

In [7]:
def enemy_attack(data, etype):
    data['player_hp'] -= etype['attack_base_hp'] * get_type_damage_modifier(data, 'player')
    data['last_actor'] = 'enemy'
    data['enemy_move_type'] = 'ATTACK'

    print("ENEMY used: ATTACK")

In [8]:
# ENEMY TURN
def state_enemy_turn(data):
    print("=== ENEMY TURN ===")
    ehp = data["enemy_hp"]
    etype = data["enemy_type"]

    # ATTACK if: attack dmg > player's hp
    if (etype['attack_base_hp'] * get_type_damage_modifier(data, 'player')) >= data['player_hp']:
        enemy_attack(data, etype)
        return "CHECK_WIN"

    # HEAL if low (probability-based)
    if ehp < (etype["base_hp"] * etype["heal_threshold"]):
        if random.random() < etype["heal_probability"]:
            data["enemy_hp"] += etype["heal_base_hp"]

            print("ENEMY used: HEAL")
            data['enemy_move_type'] = 'HEAL'
            data["last_actor"] = "enemy"
            return "CHECK_WIN"

    # default: ATTACK 
    enemy_attack(data, etype)
    return "CHECK_WIN"

In [9]:
# WIN DETECTION

def state_check_win(data):
    data["player_hp"] = round(data["player_hp"])
    hp = data["player_hp"]
    if hp < 0: hp = 0

    data["enemy_hp"] = round(data["enemy_hp"])
    ehp = data["enemy_hp"]
    if ehp < 0: ehp = 0


    print(f"Player HP: {hp}")
    print(f"Enemy  HP: {ehp}\n")

    if hp <= 0:
        print("You lose.")
        return "END"
    if ehp <= 0:
        print("You win!")
        return "END"

    # Continue cycle
    last_actor = data['last_actor']
    starting_actor = 'player' if data['is_player_first'] else 'enemy'
    non_starting_actor = 'ENEMY' if starting_actor == 'player' else 'PLAYER'

    # if turn is over: go back to move selection
    if last_actor != starting_actor:
        return "INPUT_PLAYER"
    # else: go to 2nd actor
    else:
        return f"{non_starting_actor}_TURN"


In [10]:
# END STATE
def state_end(data):
    print("=== BATTLE FINISHED ===")
    return None

In [11]:
# GAME BEGIN
STATES = {
    "START": state_start,
    "DISPLAY_INFO": state_display_info,
    "INPUT_PLAYER": state_input_player,
    "PLAYER_TURN": state_player_turn,
    "ENEMY_TURN": state_enemy_turn,
    "CHECK_WIN": state_check_win,
    "END": state_end,
}


def run_fsm():
    data = {}
    state = "START"

    while state is not None:
        state = STATES[state](data)
        

In [12]:
# Start game
run_fsm()

=== BATTLE STARTING ===
ENEMY goes first!

--PLAYER INFO--
Health: 130
Type: Grass

--ENEMY INFO--
Health: 110
Type: Water

=== ENEMY TURN ===
ENEMY used: ATTACK
Player HP: 118
Enemy  HP: 110

=== PLAYER TURN ===
YOU used: ATTACK
Player HP: 118
Enemy  HP: 80

=== ENEMY TURN ===
ENEMY used: ATTACK
Player HP: 106
Enemy  HP: 80

=== PLAYER TURN ===
YOU used: ATTACK
Player HP: 106
Enemy  HP: 50

=== ENEMY TURN ===
ENEMY used: ATTACK
Player HP: 94
Enemy  HP: 50

=== PLAYER TURN ===
YOU used: SPECIAL
Successful Counter-Attack!
Player HP: 94
Enemy  HP: 18

=== ENEMY TURN ===
ENEMY used: ATTACK
Player HP: 82
Enemy  HP: 18

=== PLAYER TURN ===
YOU used: ATTACK
Player HP: 82
Enemy  HP: 0

You win!
=== BATTLE FINISHED ===
